In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import mlflow

In [2]:
mlflow.set_tracking_uri('http://3.7.252.23:5000')
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://youtube-mlflow-campusx/728060098183646235', creation_time=1731836197994, experiment_id='728060098183646235', last_update_time=1731836197994, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
df = pd.read_csv('dataset.csv').dropna()

In [4]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic Regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2024-11-17 18:02:47,510] A new study created in memory with name: no-name-25e82eec-89f8-4c0d-97b4-8f3e741d5449
[I 2024-11-17 18:02:48,008] Trial 0 finished with value: 0.4273937856689918 and parameters: {'C': 0.006832684523415517, 'penalty': 'l1'}. Best is trial 0 with value: 0.4273937856689918.
[I 2024-11-17 18:02:49,045] Trial 1 finished with value: 0.6586345381526104 and parameters: {'C': 0.0043968332582255245, 'penalty': 'l2'}. Best is trial 1 with value: 0.6586345381526104.
[I 2024-11-17 18:02:49,955] Trial 2 finished with value: 0.6882265905728175 and parameters: {'C': 0.018019394535038844, 'penalty': 'l2'}. Best is trial 2 with value: 0.6882265905728175.
[I 2024-11-17 18:02:50,180] Trial 3 finished with value: 0.3333333333333333 and parameters: {'C': 0.0013962599030033183, 'penalty': 'l1'}. Best is trial 2 with value: 0.6882265905728175.
[I 2024-11-17 18:02:51,017] Trial 4 finished with value: 0.7269076305220884 and parameters: {'C': 0.07742644165815982, 'penalty': 'l2'}. Bes